<a href="https://colab.research.google.com/github/Azure06072005/DS317-Data-Mining-for-Enterprise/blob/main/merge_reply_user_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import necessary libraries


In [15]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate

warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

In [5]:
BASE_DIR = '/content/drive/MyDrive/DS317.Q11 nhóm 8/DataMOOCDS317'
MY_TEST_DIR = '/content/drive/MyDrive/DS317.Q11 nhóm 8/Bài tập trên lớp/Bài tập phân tích Dataset/Data/test'
df_user_course_path = '/content/drive/MyDrive/DS317.Q11 nhóm 8/DataMOOCDS317/user_course_final_ver3/user_course_v4.csv'

# User-video

In [6]:
df_user_video = spark.read.json(BASE_DIR + '/user-video.json')


In [13]:
df_user_video = spark.read.json(BASE_DIR + '/user-video.json')
# Explode the outer array in 'seq' to get each video sequence as a row
df_user_video = df_user_video.select("user_id", explode("seq").alias("video_sequence"))

# Select the video_id and explode the 'segment' array within each video sequence
df_user_video = df_user_video.select(
    "user_id",
    df_user_video["video_sequence"]["video_id"].alias("video_id"),
    explode(df_user_video["video_sequence"]["segment"]).alias("segment_details")
)

# Select the individul fields from the 'segment_details' struct
df_user_video = df_user_video.select(
    "user_id",
    "video_id",
    df_user_video["segment_details"]["start_point"].alias("start_point"),
    df_user_video["segment_details"]["end_point"].alias("end_point"),
    df_user_video["segment_details"]["speed"].alias("speed"),
    df_user_video["segment_details"]["local_start_time"].alias("local_start_time")
)

df_user_video = df_user_video.withColumn("watched_duration", round(col('end_point') - col('start_point')))
df_user_video = df_user_video.select("user_id", "video_id", "speed", "watched_duration")

In [14]:
df_user_video.show()

+-------+---------+-----+----------------+
|user_id| video_id|speed|watched_duration|
+-------+---------+-----+----------------+
|  U_112|V_1395633|  1.0|            60.0|
|  U_112|V_1395633|  1.0|            30.0|
|  U_112|V_1395633|  1.0|            30.0|
|  U_112|V_1395633| 1.25|            37.0|
|  U_112|V_1395635|  1.0|            35.0|
|  U_112|V_1395636|  1.5|            53.0|
|  U_112|V_1395639| 1.25|             6.0|
|  U_112|V_1395639| 1.25|             6.0|
|  U_112|V_6210799| 1.25|            13.0|
|  U_112|V_6210799| 1.25|           244.0|
|  U_112|V_6210799|  1.5|            53.0|
|  U_112|V_6210799|  1.5|            10.0|
|  U_112|V_6210800|  1.5|             8.0|
|  U_150|V_6224262|  2.0|           220.0|
|  U_172|V_6432959|  1.0|             5.0|
|  U_189|V_6334508|  1.0|           105.0|
|  U_189|V_6334508|  1.0|           135.0|
|  U_189|V_6334516|  1.0|             5.0|
|  U_189|V_6334519|  1.0|           840.0|
|  U_189|V_6334017|  1.0|           317.0|
+-------+--

In [17]:
df_user_video_grouped = (
    df_user_video
    .groupBy("user_id")
    .agg(
        (F.sum(F.col("watched_duration") * F.col("speed")) / F.sum(F.col("watched_duration"))).alias("avg_speed"),
        F.sum("watched_duration").alias("total_watched_duration"),
        F.collect_list("video_id").alias("video_list")
    )
)

In [18]:
df_user_video_grouped.show()

+----------+------------------+----------------------+--------------------+
|   user_id|         avg_speed|total_watched_duration|          video_list|
+----------+------------------+----------------------+--------------------+
|U_10001181|               2.0|                2878.0|[V_7386535, V_738...|
|U_10001587|1.9301075268817205|                8184.0|[V_6043067, V_604...|
| U_1000290|               1.0|                2311.0|[V_6280707, V_628...|
| U_1000454|               1.0|                 495.0|[V_6181150, V_739...|
|  U_100066|               1.0|                  20.0|         [V_6335195]|
|U_10008027|1.2237261146496816|                6280.0|[V_6043035, V_604...|
|U_10011100|               1.0|               32566.0|[V_6043065, V_604...|
|U_10012258|               2.0|               19257.0|[V_7386534, V_738...|
| U_1001314|               1.0|                  20.0|         [V_6340765]|
|U_10013620|               2.0|                3200.0|[V_7386534, V_738...|
|U_10016535|

In [19]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_uc_uv = df_user_course.join(df_user_video_grouped, on=['user_id'], how='left')
df_uc_uv.show(truncate=False)

+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+----------------------+----------+
|user_id   |course_id|certificate|start_date|end_date  |duration_days|enroll_time        |remain_day|avg_speed|total_watched_duration|video_list|
+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+----------------------+----------+
|U_1000663 |C_2328510|0.0        |2020-11-18|2021-02-28|102.0        |2020-11-27 22:41:36|93.0      |NULL     |NULL                  |NULL      |
|U_1000868 |C_947149 |1.0        |2019-12-30|2020-04-19|111.0        |2020-03-02 15:38:45|48.0      |NULL     |NULL                  |NULL      |
|U_1000883 |C_947149 |1.0        |2019-12-30|2020-04-19|111.0        |2020-03-04 08:54:01|46.0      |NULL     |NULL                  |NULL      |
|U_1000902 |C_696724 |1.0        |2019-11-18|2020-02-29|103.0        |2019-12-29 20:40:49|62.0      |NULL     |NULL         

In [20]:
df_uc_uv.count()

7008317

In [21]:
df_uc_uv.select([count(when(col(c).isNull(), c)).alias(c) for c in df_uc_uv.columns]).show()

+-------+---------+-----------+----------+--------+-------------+-----------+----------+---------+----------------------+----------+
|user_id|course_id|certificate|start_date|end_date|duration_days|enroll_time|remain_day|avg_speed|total_watched_duration|video_list|
+-------+---------+-----------+----------+--------+-------------+-----------+----------+---------+----------------------+----------+
|      0|        0|    4939141|   4939141| 4939141|      4939141|          0|   4939141|  6224092|               6223990|   6223990|
+-------+---------+-----------+----------+--------+-------------+-----------+----------+---------+----------------------+----------+



In [22]:
df_video = spark.read.json(BASE_DIR + '/video.json')
df_video.show(5,truncate=True)

+--------------------+--------------------+------------------+--------------------+------------------------------------+
|                ccid|                 end|              name|               start|                                text|
+--------------------+--------------------+------------------+--------------------+------------------------------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|             Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|
|0003DB14A14A53279...|[12.581, 17.38, 1...|             Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|    家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|
|00059EBD1371A6959...|[23.14, 194.458, ...|1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|  什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|
+--------------------+--------------------+------------------+--------------------+------------------------------------+
only

In [23]:
from pyspark.sql.functions import array_max
df_video = df_video.withColumn('video_duration', array_max('end'))

In [42]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')

In [43]:
from pyspark.sql.functions import split

# Tách cột value thành 2 cột video_id và ccid
df_video_id_ccid = df_video_id_ccid.withColumn('video_id', split(df_video_id_ccid['value'], '\t').getItem(0)) \
           .withColumn('ccid', split(df_video_id_ccid['value'], '\t').getItem(1))
df_video_id_ccid = df_video_id_ccid.select('ccid', 'video_id')
df_video_id_ccid.show(truncate=False)


+--------------------------------+---------+
|ccid                            |video_id |
+--------------------------------+---------+
|0000363DB5B6E0869C33DC5901307461|V_234845 |
|0000363DB5B6E0869C33DC5901307461|V_234876 |
|0000363DB5B6E0869C33DC5901307461|V_234907 |
|0000363DB5B6E0869C33DC5901307461|V_293392 |
|0000363DB5B6E0869C33DC5901307461|V_293445 |
|0000363DB5B6E0869C33DC5901307461|V_293494 |
|0000363DB5B6E0869C33DC5901307461|V_627142 |
|0000363DB5B6E0869C33DC5901307461|V_944346 |
|0000363DB5B6E0869C33DC5901307461|V_944377 |
|0000363DB5B6E0869C33DC5901307461|V_944408 |
|0000363DB5B6E0869C33DC5901307461|V_1386166|
|0000363DB5B6E0869C33DC5901307461|V_1479635|
|0000363DB5B6E0869C33DC5901307461|V_1494506|
|0000363DB5B6E0869C33DC5901307461|V_1494536|
|0000363DB5B6E0869C33DC5901307461|V_1494566|
|0000363DB5B6E0869C33DC5901307461|V_1494614|
|0000363DB5B6E0869C33DC5901307461|V_1494625|
|0000363DB5B6E0869C33DC5901307461|V_1494656|
|0000363DB5B6E0869C33DC5901307461|V_1570352|
|0000363DB

In [44]:
df_video = df_video.join(df_video_id_ccid, on=['ccid'], how='left')
df_video.show(5)

+--------------------+--------------------+--------------+--------------------+---------------------------------+--------------+--------+--------+--------+---------+
|                ccid|                 end|          name|               start|                             text|video_duration|video_id|video_id|video_id| video_id|
+--------------------+--------------------+--------------+--------------------+---------------------------------+--------------+--------+--------+--------+---------+
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448343|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448628|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448757|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84,

In [40]:
df_video = df_video.select("video_duration", "video_id")

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `video_id` is ambiguous, could be: [`video_id`, `video_id`, `video_id`].

In [ ]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_course.show(5,truncate=True)

In [ ]:
df_video.printSchema()

In [ ]:
df_video.show(5)

In [ ]:
df_video_duration = df_video.select('video_id', 'duration')
df_video_duration.show(5)

In [ ]:
df_user_video.show(5)

In [ ]:
df_user_video = df_user_video.join(df_video_duration, on=['video_id'], how='left')
df_user_video.show(5)

In [ ]:
df_uc_uv = df_uc_uv.join(df_video_duration, on = ["video_id"], how = 'left')
df_uc_uv.show(5)

In [ ]:
from pyspark.sql.functions import col, isnan, when, count
df_uc_uv.count()
df_uc_uv.select([count(when(col(c).isNull(), c)).alias(c) for c in df_uc_uv.columns]).show()

In [ ]:
df_user_course.show(5)

In [ ]:
# df_uc_uv.write.mode('overwrite').parquet(MY_TEST_DIR)